<a href="https://colab.research.google.com/github/OSEUK/Sketch2Dream/blob/master/Sd_scribble_imageGeneration_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 본인 huggingface_hub 토큰으로 로그인
from huggingface_hub import login
login()

In [2]:
# 1. 필요한 라이브러리 설치
!pip install diffusers transformers accelerate opencv-python safetensors
!pip install flask-ngrok flask pyngrok
!pip install flask-cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.2 MB/s eta 0:00:00


In [4]:
# 2. 모델 및 라이브러리 임포트
from flask import Flask, request, jsonify
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import cv2
from PIL import Image
import numpy as np
import io
from io import BytesIO
from pyngrok import ngrok
from flask_cors import CORS
import base64


# ngrok 터널 열기
ngrok.set_auth_token("YOUR_NGROK_TOKEN")  # 이 부분에 실제 Ngrok 토큰을 추가하세요
ngrok_tunnel = ngrok.connect(5000)
public_url = ngrok_tunnel.public_url  # 정확한 URL만 추출
print("ngrok URL:", public_url)

# Flask 앱 설정
app = Flask(__name__)
# CORS 설정: 모든 경로에 대해 모든 출처에서의 요청 허용
CORS(app, resources={r"/*": {"origins": "*"}})


# 3. ControlNet scribble 모델 로드
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-scribble", torch_dtype=torch.float16
)

# 4. Stable Diffusion 모델과 ControlNet 파이프라인 설정
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", controlnet=controlnet, torch_dtype=torch.float16
)

# 5. GPU 사용 설정
pipe.to("cuda")

# 스케줄러 설정 (더 나은 품질을 위해)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

@app.route('/process_image', methods=['POST','OPTIONS'])
def process_image():
    # 이미지 수신
    # 이미지 파일 수신
    if request.method == 'OPTIONS':
        return jsonify({'status': 'Preflight check successful'}), 200  # Preflight 요청에 대한 응답

    data = request.json

    input_text = data['text']  # 사용자의 텍스트 데이터
    image_data = data['image']  # 이미지 데이터 (Base64 혹은 Data URL 형식)

    # Base64 이미지 데이터가 제대로 수신되었는지 확인
    print(f"Received Base64 Image Data (first 100 chars): {image_data[:100]}")
    # Base64 이미지를 디코딩하여 PIL 이미지로 변환
    try:
        # "data:image/png;base64,..." 형식으로 되어있다면 쉼표 뒤의 데이터만 가져옴
        image_data = image_data.split(",")[1]
        image_bytes = base64.b64decode(image_data)
        image = Image.open(BytesIO(image_bytes))

    except Exception as e:
        return jsonify({'error': 'Failed to process image', 'message': str(e)}), 400

    # 프롬프트 설정
    prompt = (
        f"Generate a high-resolution, realistic photograph of {input_text} based on this sketch. "
        "The image should feature realistic lighting, sharp details, and a vivid background."
    )
    negative_prompt = "sketch, drawing, cartoon, unrealistic, low quality, blurry, oversaturated"

    # 이미지 변환
    with torch.no_grad():
        result = pipe(prompt=prompt, image=image, negative_prompt=negative_prompt)

    # 결과 이미지
    result_image = result.images[0]

    # 입력 이미지가 제대로 왔는 지 확인
    input_path = '/content/input_image.png'
    image.save(input_path)
    # 생성된 이미지 파일로 저장
    output_path = '/content/output_image.png'
    result_image.save(output_path)

    # 이미지를 메모리로 저장
    buffered = BytesIO()
    result_image.save(buffered, format="PNG")

    # 이미지 데이터를 Base64로 인코딩
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

    # 변환된 이미지 URL을 클라이언트로 반환
    image_url = f"{public_url}{output_path}"

    return jsonify({'image_data': img_str})


if __name__ == '__main__':
    app.run()



ngrok URL: https://afa6-34-87-70-4.ngrok-free.app


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Sep/2024 01:57:28] "OPTIONS /process_image HTTP/1.1" 200 -


Received Base64 Image Data (first 100 chars): data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCI


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [09/Sep/2024 01:57:40] "POST /process_image HTTP/1.1" 200 -


In [ ]:
# 계속하다가 뜬금없는 에러가 뜬다면 한번쯤 해보기
!kill $(pgrep ngrok)